если у меня есть список из 10 разных слов, путон нашел их в тексте и протегал мне массив?

In [1]:
import pandas as pd
from pymystem3 import Mystem

# Вариант. Просто перебор и поиск по словам

In [2]:
try:
    df = (
    pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') 
)
except FileNotFoundError:
    print("File not found.")
except pd.errors.EmptyDataError:
    print("No data")
except pd.errors.ParserError:
    print("Parse error")
except Exception:
    print("Some other exception")

- **Нам интересен столбец крайний самый `Purpose`**

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


- **Получается обрезаешь чуть окончания, и он уже ищет в столбце тебе совпадения и выдает новый столбец с категориями.**

In [4]:
m = Mystem()
def purpose_group (purpose):
    lemmas = m.lemmatize(purpose)
    for i in lemmas:
        if 'свадьб' in i:
            return 'проведение свадьбы'
        elif 'образов' in i:
            return 'получение образования'
        elif 'недвиж' in i:
            return 'операции с недвижимостью'
        elif 'жил' in i:
            return 'операции с недвижимостью'
        elif 'автомобил' in i:
            return 'операции с автомобилем'

## Результат

In [5]:
df['purpose_category'] = df['purpose'].apply(purpose_group)
df['purpose_category'].value_counts()

операции с недвижимостью    10840
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

# Вариант с созданием в ручную(либо+ML чтоб все автоматизировать) словарем

In [6]:
try:
    s_df = pd.read_csv('https://code.s3.yandex.net/datasets/ecommerce_dataset.csv') 

except FileNotFoundError:
    print("File not found.")
except pd.errors.EmptyDataError:
    print("No data")
except pd.errors.ParserError:
    print("Parse error")
except Exception:
    print("Some other exception")

In [7]:
del s_df['date']
del s_df['customer_id']
del s_df['order_id']
del s_df['quantity']
del s_df['price']

- **Можешь заметить в столбце `Product` наименования товаров всяких разных, я брал обрезал 2 слова, чтобы лемматизация не грузила сильно пекарню, можно и без нее, с # покажу без нее, и с ней**

In [8]:
s_df.head()

,product
0,"Комнатное растение в горшке Алое Вера, d12, h30"
1,"Комнатное растение в горшке Кофе Арабика, d12,..."
2,Радермахера d-12 см h-20 см
3,Хризолидокарпус Лутесценс d-9 см
4,Циперус Зумула d-12 см h-25 см


In [9]:
# Оставил 2 слова первых и 
s_df["new_one"] = s_df["product"].str.split().str[0]
s_df["new_two"] = s_df["product"].str.split().str[1]

# соединил их между собой
s_df['product_new'] = s_df['new_one'] + ' ' + s_df['new_two']

# Удалил 2 стобца лишних
del s_df['new_one']
del s_df['new_two']
s_df

,product,product_new
0,"Комнатное растение в горшке Алое Вера, d12, h30",Комнатное растение
1,"Комнатное растение в горшке Кофе Арабика, d12,...",Комнатное растение
2,Радермахера d-12 см h-20 см,Радермахера d-12
3,Хризолидокарпус Лутесценс d-9 см,Хризолидокарпус Лутесценс
4,Циперус Зумула d-12 см h-25 см,Циперус Зумула
...,...,...
6732,"Таз пластмассовый 40,0 л пищевой овальный (Иж)...",Таз пластмассовый
6733,Сумка-тележка 2-х колесная Gimi Argo синяя,Сумка-тележка 2-х
6734,Герань домашняя (пеларгония зональная) цветуща...,Герань домашняя
6735,"Таз пластмассовый со стиральной доской (Иж), 1...",Таз пластмассовый


- **Импорт библиотеки, функция короче дает тебе уникальное количество слов.**

In [11]:
from collections import Counter
all_words = []
for msg in s_df['product_new'].unique():
    words = str(msg).lower().strip().split(' ')
    all_words.extend(words)
            
count = Counter(all_words)
count.most_common()

[('для', 70),
 ('муляж', 43),
 ('рассада', 33),
 ('скатерть', 23),
 ('набор', 21),
 ('петуния', 21),
 ('с', 16),
 ('в', 15),
 ('коврик', 14),
 ('стремянка', 13),
 ('вешалка', 12),
 ('кружка', 12),
 ('пеларгония', 11),
 ('микс', 11),
 ('d-12', 10),
 ('d-9', 10),
 ('роза', 10),
 ('салатник', 9),
 ('фуксия', 8),
 ('дыня', 8),
 ('алюминиевая', 7),
 ('сумка-тележка', 7),
 ('нож', 7),
 ('ведро', 7),
 ('огурец', 7),
 ('вербена', 7),
 ('ложка', 6),
 ('банка', 6),
 ('d-13', 6),
 ('хризантема', 6),
 ('фиалка', 6),
 ('швабра', 6),
 ('ампельная,', 6),
 ('webber', 6),
 ('космея', 6),
 ('капуста', 6),
 ('калибрахоа', 6),
 ('базилик', 6),
 ('сушилка', 5),
 ('пластмассовый', 5),
 ('корзина', 5),
 ('хозяйственная', 5),
 ('перчатки', 5),
 ('на', 5),
 ('махровая', 5),
 ('полотенце', 5),
 ('цикламен', 5),
 ('салфетка', 5),
 ('щетка', 5),
 ('насадка', 5),
 ('мята', 5),
 ('кастрюля', 5),
 ('арбуз', 5),
 ('доска', 4),
 ('гладильная', 4),
 ('клубника', 4),
 ('алюминиевый', 4),
 ('ёрш', 4),
 ('чайник', 4),
 ('

- **Дальше жопа, в ручную необхоимо перебрать все слова сверху, у меня получилось категории Сад, Растения, Кухня, Хоз.Товары, все что не войдет в словарь попадет в другое.**

In [12]:
dictionary = {'муляж': 'сад', 'рассада': 'растения', 'скатерть': 'кухня', 'петуния': 'растения',
              'вешалка': 'хоз.товары', 'коврик': 'хоз.товары', 'пеларгония': 'растения', 'стремянка': 'хоз.товары',
              'кружка': 'кухня', 'салатник': 'кухня', 'роза': 'растения', 'фуксия': 'растения', 'дыня': 'растения',
              'огурец': 'растения', 'вербена': 'растения', 'сушилка': 'хоз.товары', 'щетка': 'хоз.товары', 'ложка': 'кухня',
              'нож': 'кухня', 'банка': 'хоз.товары', 'хризантема': 'растения', 'швабра': 'хоз.товары', 'ведро': 'хоз.товары', 'космея': 'растения', 'капуста': 'растения', 
              'калибрахоа': 'растения', 'корзина': 'хоз.товары', 'перчатки': 'хоз.товары', 'полотенце': 'хоз.товары', 'салфетка': 'хоз.товары', 
              'фиалка': 'растения', 'мята': 'растения', 'кухня': 'растения', 'базилик': 'растения', 'арбуз': 'растения', 'кастрюля': 'кухня', 
              'доска': 'хоз.товары', 'клубника': 'растения', 'чайник': 'кухня', 'мыло': 'хоз.товары', 'коробка': 'хоз.товары', 'стакан': 'хоз.товары',
              'примула': 'растения', 'гвоздика': 'растения', 'термокружка': 'кухня', 'тимьян': 'растения', 'томат': 'растения', 'цинния': 'растения', 'растение': 'растения',
              'чехол': 'хоз.товары', 'сумка': 'хоз.товары', 'тележка': 'сад', 'тарелка': 'кухня', 'эхеверия': 'растения', 'контейнер': 'хоз.товары', 'простыня': 'хоз.товары',
              'штора': 'хоз.товары', 'цикламен': 'растения', 'миксер': 'кухня', 'подрукавник': 'хоз.товары', 'паста': 'кухня', 'петля': 'хоз.товары', 'лаванда': 'растения',
              'дендробиум': 'растения', 'настурция': 'растения', 'скребок': 'хоз.товары', 'алиссум': 'растения', 'календула': 'растения', 'флокс': 'растения', 'покрывало': 'хоз.товары',
              'весы': 'кухня', 'фаленопсис': 'растения', 'миска': 'кухня', 'совок': 'хоз.товары', 'хлебница': 'кухня', 'спатифиллум': 'растения', 'лавр': 'растения', 'лобелия': 'растения',
              'патиссон': 'растения', 'петрушка': 'растения', 'шеффлера': 'растения', 'таз': 'хоз.товары', 'крышка': 'хоз.товары', 'сметка': 'хоз.товары', 'эвкалипт': 'растения',
              'цветок': 'растения', 'кувшин': 'сад', 'толкушка': 'кухня', 'крючок': 'хоз.товары', 'калатея': 'растения', 'плечики': 'хоз.товары', 'подушка': 'сад', 'глоксиния': 'растения', 
              'азалия': 'растения', 'гортензия': 'растения', 'тыквы': 'растения', 'гортензия': 'хоз.товары', 'тыквы': 'растения', 'кориандр': 'растения', 'аспарангус': 'растения', 
              'сверло': 'хоз.товары', 'антуриум': 'растения', 'молодило': 'растения', 'гербера': 'растения', 'карниз': 'хоз.товары', 'веник': 'хоз.товары', 'ящик': 'сад',
              'подголовник': 'хоз.товары', 'розмарин': 'растения', 'утюг': 'хоз.товары', 'баклажан': 'растения', 'морковь': 'растения', 'шнур': 'хоз.товары', 'урна': 'хоз.товары', 
              'рыбочистка': 'кухня', 'терка': 'кухня', 'лопатка': 'сад', 'бальзамин': 'растения', 'крассула': 'растения', 'лоток': 'сад', 'мантоварка': 'кухня', 'термометр': 'хоз.товары',
              'лапчатка': 'растения', 'диффенбахия': 'одеяло', 'хоз.товары': 'растения', 'пуансеттия': 'растения', 'этажерка': 'хоз.товары', 'термос': 'кухня', 'разделочная': 'кухня', 
              'томата': 'растения', 'аквилегия': 'растения', 'земляника': 'растения', 'седум': 'растения', 'бакопа': 'растения', 'виола': 'растения', 'колокольчик': 'растения', 
              'прищепка': 'хоз.товары', 'астра': 'растения', 'фатсия': 'растения', 'кабачок': 'растения', 'монарда': 'растения', 'камнеломка': 'растения', 'лантана': 'растения', 
              'соковарка': 'кухня', 'радермахера': 'растения', 'хризолидокарпус': 'растения', 'лутесценс': 'растения', 'циперус': 'растения', 'зумула': 'растения', 'юкка': 'растения',
              'груша': 'растения', 'перец': 'растения', 'персики': 'растения', 'томат': 'растения', 'черешня': 'растения', 'вилка': 'кухня', 'герань': 'растения',
              'ерш': 'хоз.товары', 'котел': 'кухня', 'суккулент': 'растения', 'дозатор': 'сад', 'виноград': 'растения', 'вишня,': 'растения', 'лестница': 'хоз.товары',
              'ковер': 'хоз.товары', 'яблоко': 'растения', 'кампанула': 'растения', 'противень': 'кухня', 'хлорофитум': 'растения', 'вкладыши': 'хоз.товары', 'tepмокружка': 'кухня',
              'сад': 'хоз.товары', 'сантолина': 'растения', 'tepмокружка': 'кухня', 'фреза': 'хоз.товары', 'короб': 'сад', 'камушки': 'сад', 'масленка': 'кухня',
              'овощеварка': 'кухня', 'линейка': 'хоз.товары', 'салфетница': 'кухня', 'альбука': 'растения', 'спиралис': 'растения', 'овата': 'растения', 'артемизия': 'растения',
              'полынь': 'растения', 'стойка': 'сад', 'желудь': 'растения', 'каланхое': 'растения', 'каланхое': 'растения', 'каландива': 'растения', 'акаулис': 'растения',
              'держатель': 'сад', 'пробка': 'хоз.товары', 'измельчитель': 'кухня', 'вешалки': 'хоз.товары', 'губка': 'кухня', 'калоцефалус': 'растения', 'каллуна': 'растения', 
              'капсикум': 'растения', 'мимоза': 'растения', 'соланум': 'растения', 'патио': 'сад', 'бархатцы': 'растения', 'шило': 'хоз.товары', 'ковш': 'кухня', 
              'ножницы': 'сад', 'кисточка': 'кухня', 'картофелемялка': 'кухня', 'лобулярия': 'растения', 'гореция': 'растения', 'друммонда': 'растения', 'цинерария': 'растения',  
              'эшшольция': 'растения', 'рассекатель': 'кухня', 'сетка': 'сад', 'подсолнечник': 'растения', 'подставка': 'хоз.товары', 'пуанситея': 'растения', 'пресс': 'кухня', 
              'термостакан': 'кухня', 'гиностемма': 'растения', 'ель': 'растения', 'кипарисовик': 'растения', 'лавсона': 'растения', 'аптения': 'растения', 'мыльница': 'хоз.товары', 
              'грандифлора': 'растения', 'вантуз': 'хоз.товары', 'скиммия': 'растения', 'спренжери': 'сад', 'электроштопор': 'кухня', 'блюдце': 'кухня', 'вафельная': 'кухня',
              'фиттония': 'растения', 'беатрис': 'растения', 'бульонница': 'кухня', 'алоэ': 'растения', 'ключница': 'хоз.товары', 'пароварка': 'кухня', 'фикус': 'растения', 
              'соль': 'кухня', 'просеиватель': 'кухня', 'кодонанта': 'растения', 'цикламен,': 'растения', 'гимнокалициум': 'растения', 'крепеж': 'хоз.товары', 'салфетки': 'кухня', 
              'кроката': 'растения', 'эхинокактус': 'растения', 'грузони': 'растения', 'фал': 'сад', 'орехоколка': 'кухня', 'штанга': 'сад', 'маска': 'сад', 
              'химера': 'растения', 'крокусы': 'растения', 'гипсофила': 'растения', 'горох': 'растения', 'амброзия': 'растения', 'незабудка': 'растения', 'укроп': 'растения', 
              'нефролепис': 'сад', 'калла': 'растения', 'тюльпан': 'растения', 'гиацинт': 'растения', 'скалка': 'кухня', 'рукав': 'кухня', 'чабер': 'растения', 
              'венчик': 'кухня', 'сито': 'кухня', 'газания': 'растения', 'овсянница': 'растения', 'тортница': 'кухня', 'георгина': 'растения', 'смолевка': 'растения', 
              'травянка': 'растения', 'остеоспермум': 'растения', 'котовник': 'растения', 'эхинацея': 'растения', 'ясколка': 'растения', 'блюдо': 'кухня', 'лилейник': 'растения', 
              'папоротник': 'растения', 'страусник': 'растения', 'платикодон': 'растения', 'энотера': 'растения', 'цитрофортунелла': 'растения', 'кумкват': 'растения', 'тряпкодержатель': 'хоз.товары', 
              'пиретрум': 'растения', 'агератум': 'растения', 'колеус': 'растения', 'тагетис': 'растения', 'цинерания': 'растения', 'муррайя': 'растения', 'портулак': 'растения',
              'аргирантерум': 'растения', 'свитуния': 'растения', 'целозия': 'растения', 'брокколи': 'растения', 'драцена': 'растения', 'фрагранс': 'растения', 'хоста': 'растения', 'гайлардия': 'растения', 
              'нивянник': 'растения', 'ромашка': 'растения', 'рудбекия': 'растения', 'очиток': 'растения', 'барвинок': 'растения', 'элатиор': 'растения', 'сциндапсус': 'растения', 'бузульник': 'растения', 
              'арендса': 'растения', 'солидаго': 'растения', 'физостегия': 'растения', 'бадан': 'растения', 'эмпилабль': 'кухня', 'сотейник': 'кухня', 'бидон': 'кухня', 'лаватера': 'растения', 'монт': 'растения', 
              'анемона': 'растения', 'буддлея': 'растения', 'валериана': 'растения', 'змееголовник': 'растения', 'кольраби': 'растения', 'любисток': 'растения', 'удалец': 'растения', 'шалфей': 'растения', 
              'корзинка': 'сад', 'ваза': 'сад', 'вигна': 'растения', 'лилиана': 'растения', 'тыква': 'растения', 'хамедорея': 'растения', 'зверобой': 'растения', 'мединилла': 'растения', 'импатиенс': 'растения',
              'клен': 'растения', 'веревка': 'сад', 'мешок': 'хоз.товары', 'замиокулькас': 'растения', 'сахарница': 'кухня', 'пьезозажигалка': 'кухня', 'соковыжималка': 'кухня', 'салат': 'растения', 'рукола': 'растения',
              'пакет': 'хоз.товары', 'тряпка': 'хоз.товары', 'половник': 'кухня'}

In [13]:
m = Mystem()
def purpose_group(purpose_category):
        purpose_list = m.lemmatize(purpose_category)
        for name in purpose_list: 
            if name in (dictionary):
                purpose_string =  dictionary[name]
                return purpose_string
            else: purpose_string = 'другое'
            return purpose_string 

## Результат

In [14]:
s_df['product_group'] = s_df['product_new'].apply(purpose_group)
s_df

,product,product_new,product_group
0,"Комнатное растение в горшке Алое Вера, d12, h30",Комнатное растение,другое
1,"Комнатное растение в горшке Кофе Арабика, d12,...",Комнатное растение,другое
2,Радермахера d-12 см h-20 см,Радермахера d-12,другое
3,Хризолидокарпус Лутесценс d-9 см,Хризолидокарпус Лутесценс,растения
4,Циперус Зумула d-12 см h-25 см,Циперус Зумула,растения
...,...,...,...
6732,"Таз пластмассовый 40,0 л пищевой овальный (Иж)...",Таз пластмассовый,хоз.товары
6733,Сумка-тележка 2-х колесная Gimi Argo синяя,Сумка-тележка 2-х,хоз.товары
6734,Герань домашняя (пеларгония зональная) цветуща...,Герань домашняя,растения
6735,"Таз пластмассовый со стиральной доской (Иж), 1...",Таз пластмассовый,хоз.товары


- **В целом если не убирать слова лишние, но есть время дрочиться со словарем, то можно сразу найти лемы(1) и уже функцию писать(2)**

In [15]:
#unique_product = ecommerce['product'].unique() 

# Выделим леммы в значениях столбца с наименованием товара, которые пригодиться для создания словаря, 
# который будем исползать для категоризации товара.
#m = Mystem()
#string = '; '.join(unique_product) # сохраным результат в переменной string
#lemmas = m.lemmatize(string) # сохраным результат в переменной lemmas

In [16]:
#category_dictionary = {} #  словарик короче

In [19]:
#def categorize_product(product):
    #""" 
    #Возвращает категорию по значению ‘product’ в датасете ‘ecommerce’, 
    #используя словарь с категориями ‘category_dictionary’.
    #""" 
    #for category, items_of_list in category_dictionary.items():
        #for item in items_of_list:
            #if item in product:
                #return category
    #return category

In [18]:
#s_df['category'] = s_df['product'].apply(categorize_product) # применим функцию выше к датасету

#  Вариант с библиотекой Re

In [20]:
import re

In [21]:
try:
    data_rest = (
    pd.read_csv('https://code.s3.yandex.net/datasets/rest_data.csv') 
)
except FileNotFoundError:
    print("File not found.")
except pd.errors.EmptyDataError:
    print("No data")
except pd.errors.ParserError:
    print("Parse error")
except Exception:
    print("Some other exception")

- **Нам интересен столбец address, короче оттуда выдернуть все лишнее и оставить лишь нужную инфу, быстренько приведем значения к нижнему регистру и начнем**

In [22]:
del data_rest['id']
del data_rest['object_name']
del data_rest['chain']
del data_rest['object_type']
del data_rest['number']

data_rest['address'] = data_rest['address'].str.lower()
data_rest

,address
0,"город москва, улица егора абакумова, дом 9"
1,"город москва, улица талалихина, дом 2/1, корпус 1"
2,"город москва, абельмановская улица, дом 6"
3,"город москва, абрамцевская улица, дом 1"
4,"город москва, абрамцевская улица, дом 9, корпус 1"
...,...
15361,"город москва, 3-й крутицкий переулок, дом 18"
15362,"город москва, улица земляной вал, дом 33"
15363,"город москва, улица земляной вал, дом 33"
15364,"город москва, поселение московский, киевское ш..."


- **Как можешь заметить, собрали слова необходимые, с помощью Re передали на стобец address и вырезали нужную информацию**

In [23]:
words = ['улица','ул','переулок','шоссе','проспект','площадь','проезд',
         'село','аллея','бульвар','набережная','тупик','линия', 'квартал']

str_pat = r".*,\s*\b([^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(words))

data_rest["street"] = data_rest["address"].str.extract(str_pat, flags=re.I)

## Результат

In [24]:
data_rest.head()

,address,street
0,"город москва, улица егора абакумова, дом 9",улица егора абакумова
1,"город москва, улица талалихина, дом 2/1, корпус 1",улица талалихина
2,"город москва, абельмановская улица, дом 6",абельмановская улица
3,"город москва, абрамцевская улица, дом 1",абрамцевская улица
4,"город москва, абрамцевская улица, дом 9, корпус 1",абрамцевская улица
